In [11]:
import numpy as np
from sympy import *
from math import floor
import time

In [12]:
#Useful functions for both Hypercubes and general Hamming graphs
def read_sets(filename):
    #Read in the sets of a file
    results = []
    with open(filename,'r') as f:
        for line in f.read().splitlines():
            results.append(line.split(','))
    return(results)

def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    z = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*z
    return(list(lin_fcns))

In [13]:
#Hypercube functions
def apply_automorphism(R):
    #Applies automorphism so that r1 = 0^k (Not useful for non-hypercubes)
    position = [i for i,r in enumerate(R[0]) if r == '0']
    R_new = [r for r in R]
    for j,r in enumerate(R_new):
        temp = list(r)
        for i in position:
            if temp[i] == '1':
                temp[i] = '0'
            else:
                temp[i] = '1'
        R_new[j] = ''.join(temp)
    return(R_new)

def hypercube_matrix(R):
    A = np.zeros((len(R),len(R[0])))
    for i,r in enumerate(R):
        for j,letter in enumerate(r):
            A[i,j] = int(letter)
    return(A)

def hypercube_polys(k):
    variable_string = ''
    for i in range(1,k+1):
        variable_string = variable_string +'z{},'.format(i)
    variable_string = variable_string[:-1]
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        P.append(z[i]*(z[i]-1)*(z[i]+1))
        f = f + z[i]**2
    fs = []
    for i in range(floor(k/2)):
        fi = f-2*(i+1)
        fs.append(fi)
    return(P,fs,z)

def check_hcube_resolving(R,k):
    #Convert R so that r1 = 1^k
    R = apply_automorphism(R)
    #Create matrix A from automorphized R
    A = hypercube_matrix(R)
    #Get polynomial functions and variables for H_k,2
    P,fs,z = hypercube_polys(k)
    #Get linear functions from A matrix
    lin_fcns = make_linearEqns(A,z)
    #Get Grobner basis of P and linear functions
    G = groebner(P+lin_fcns,order = 'lex')
    for i,fi in enumerate(fs):
        #Compute Grobner basis of G+fi
        Gi = groebner(list(G)+[fi],order = 'lex')
        #Solutions iff Gi neq 1, if Gi neq 1 then R is not resolving
        if not (list(Gi) == [1]):
            return False
    return True
    

In [23]:
def OneHot(R,k,a,alphabet = None):
    #Converts list of strings to list of one-hot encodings
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    encodings = []
    for r in R:
        encoding = np.zeros((a,k))
        for i in range(k):
            for j,letter in enumerate(alphabet):
                if r[i] == letter:
                    encoding[j,i] = 1
        encodings.append(encoding)
    return(encodings)

def create_polys(k,a):
    #Setup polynomial system for H_k,a without knowing what R is
    variable_string = 'z1'
    for i in range(1,k*a):
        variable_string = variable_string +',z{} '.format(i+1)
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        func = 0
        #func2 = 0
        for j in range(0,a):
            P.append(z[i*a+j]*(z[i*a+j]-1)*(z[i*a+j]+1)) #1st condition zi*(zi-1)*(zi+1)
            func= func + z[i*a+j]**2
            #func2 = func2 + z[i*a+j]
            f = f + z[i*a+j]**2 #f so that sum(zi**2) neq 0
        func = (2-func)*(func)
        P.append(func)
        #P.append(func2)
    
    fs = [f-2*i for i in range(1,k+1)]
    return(P,fs,z)
    
def make_matrix(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    for i in range(k):
        added_row = np.zeros(k*a)
        for j in range(a):
            #This is the 2nd condition where sum(zi)_i*a+1^i*a+a = 0
            added_row[i*a+j] = 1
        temp.append(list(added_row))
    return(np.array(temp,dtype = int))

def check_resolving(R,k,a,alphabet = None):
    (P,fs,z) = create_polys(k,a)
    OH_encodedR = OneHot(R,k,a)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    G = groebner(P+lin_fcns,order = 'lex')
    for fi in fs:
        Gi = groebner(list(G)+[fi],order = 'lex')
        if not (list(Gi) == [1]):
            return False
    return True


In [16]:
#Simplified Hypercube approach test
sets = read_sets('Test_Sets/H_2_8.txt')
is_resolving = []
for i,R in enumerate(sets):
    resolve = check_hcube_resolving(R,8)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving


In [17]:
#Checking if general approach produces same results as above
sets = read_sets('Test_Sets/H_2_8.txt')
is_resolving = []
for i,R in enumerate(sets):
    resolve = check_resolving(R,8,2)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

Set 1 is not resolving
Set 2 is not resolving
Set 3 is resolving
Set 4 is resolving
Set 5 is not resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving


In [18]:
#Simplified Hypercube approach test
sets = read_sets('Test_Sets/H_2_12.txt')
is_resolving = []
for i,R in enumerate(sets):
    resolve = check_hcube_resolving(R,12)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is not resolving
Set 8 is not resolving


In [19]:
#Checking if general approach produces same results as above
sets = read_sets('Test_Sets/H_2_12.txt')
is_resolving = []
for i,R in enumerate(sets):
    k = len(R[0])
    resolve = check_resolving(R,k,2)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is not resolving
Set 8 is not resolving


In [24]:
#Using NON-IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_3_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,5,3)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1

avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Set 1 is resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 1.35 seconds
Average time: 0.17 seconds


In [9]:
sets = read_sets('Test_Sets/H_4_4.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,4,4)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 2.68 seconds
Average time: 0.33 seconds


In [11]:
sets = read_sets('Test_Sets/H_4_5.txt')
is_resolving = []
for i,R in enumerate(sets):
    resolve = check_resolving(R,5,4)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

#8,4,5,1
#This misses Set 1 as not resolving

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\llair\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-6d1ad1a39906>", line 4, in <module>
    resolve = check_resolving(R,5,4)
  File "<ipython-input-4-f1416e11bb5e>", line 55, in check_resolving
    G = groebner(P+lin_fcns,order = 'lex')
  File "C:\Users\llair\Anaconda3\lib\site-packages\sympy\polys\polytools.py", line 6747, in groebner
    return GroebnerBasis(F, *gens, **args)
  File "C:\Users\llair\Anaconda3\lib\site-packages\sympy\polys\polytools.py", line 6786, in __new__
    G = _groebner(polys, ring, method=opt.method)
  File "C:\Users\llair\Anaconda3\lib\site-packages\sympy\polys\groebnertools.py", line 45, in groebner
    G = _groebner(seq, ring)
  File "C:\Users\llair\Anaconda3\lib\site-packages\sympy\polys\groebnertools.py", line 250, in _buchberger
    ht = normal(f[ig], G - set([ig]))
  File "C:\Users\lla

KeyboardInterrupt: 

In [ ]:
sets = read_sets('Test_Sets/H_4_6.txt')
is_resolving = []
for i,R in enumerate(sets):
    resolve = check_resolving(R,6,4)
    is_resolving.append(resolve)
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))

